In [1]:
from pathlib import Path

import pandas as pd

# The importance of sandboxes

I've worked with SQL regularly over the past decade, but over the first half of that decade (and despite extensive formal knowledge of normalization, data modeling, and other database-related topics) I essentially only used SQL to extract data and use Python or R to do analysis. I only had access to production databases and I didn't know enough about how database management system work to risk experiments, and as a result, I learned at a glacial pace. But in 2018, I stumbled into a project interesting enough to motivate me to install PostGIS on a personal machine and freed from the fear of accidentally taking down a production system (and the power of a superuser), I was able to lift up the hood and see how the machine worked, and this enabled me to learn and build with PostgreSQL + PostGIS far faster.

In this post, I show how to use Docker to set up a PostGIS database and experiment with it.


In [2]:
#| code-fold: true
#| code-summary: The full contents of our Dockerfile

!cat db_context/Dockerfile

FROM postgis/postgis:15-3.3

And this is our `docker-compose.yml` file. The `docker-compose.yml` file defines the services, and any networks, volumes, configs, and/or secrets/environment variables your system needs to work. The file below defines one service (named `postgis`) and one volume (named `sandbox_postgis_data`).

In [3]:
#| code-fold: true
#| code-summary: The contents of our docker-compose.yml

!cat docker-compose.yml

version: '3.9'

services:
  postgis:
    image: sandbox_postgis:15.3.3
    build:
      context: ./db_context
      dockerfile: Dockerfile
    ports:
      - 54321:5432
    environment:
      POSTGRES_DB: db_name
      POSTGRES_USER: db_username
      POSTGRES_PASSWORD: db_password
    volumes:
      - sandbox_postgis_data:/var/lib/postgresql/data

volumes:
  sandbox_postgis_data:


## Decoding this `docker-compose.yml` file

The `postgis` service has five elements: `image`, `build`, `ports`, `environment`, and `volumes`.

The `image` and `build` elements define the docker image to use; `image` defines both the name ("sandbox_postgis") and tag ("15.5.3") for the docker image, and `build` defines the Dockerfile to build into an image as well as the context to build into the docker image.

The `port` element defines a connection from the host machine into the 

PostgreSQL databases listen to port number 5432 by default, but I already have a postgres database connected to port 5432 on my host machine, so I've used the **ports** section to indicate port 5432 inside the container should be connected to port 54321 on the host system.
 
 Docker containers are designed to be a replicable instance of an image, so when you shut down your application, your containers are removed and new ones are created next time you start it up. This is great for reproducibility (you always get a new, clean instance based on your image), but you don't want the data you ingest into your database to get wiped every time you shut down your system, so you can define a persistent **volume** that will live on in the host system. And when the postgres database intiailly starts up, the database also creates a superuser using the credentials we pass in as environment variables `POSTGRES_DB`, `POSTGRES_USER`, and `POSTGRES_PASSWORD`.

The first time you build the image defined by your Dockerfile, docker will read your Dockerfile(s), download all layers of the base image(s) (defined in lines starting with `FROM `), process each subsequent instruction into a layer, cache layers, and then bind the layers into an image. This is a template for creating containers.

The second time you run this command (assuming no changes have been made to the `docker-compose.yml` file, Dockerfile(s), or any other files the Dockerfile references), all layers will just be pulled from cache, producing much smaller output (like what's shown below).

In [4]:
!docker compose build

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/4)                                                         
 => [postgis internal] load build definition from Dockerfile               0.0s
 => => transferring dockerfile: 64B                                        0.0s
 => [postgis internal] load .dockerignore                                  0.0s
 => => transferring context: 2B                                            0.0s
 => [postgis internal] load metadata for docker.io/postgis/postgis:15-3.3  0.1s
 => [postgis auth] postgis/postgis:pull token for registry-1.docker.io     0.0s
[+] Building 0.3s (3/4)                                                         
 => [postgis internal] load build definition from Dockerfile               0.0s
 => => transferring dockerfile: 64B                                        0.0s
 => [postgis internal] load .dockerignore                                  0.0s
 => => transferring context: 2B      

In [6]:
!docker ps -f name=sandbox-postgis

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


`docker ps` shows running containers, and the `-f name=...` option allows us to filter to running containers with a name containing the entered string. Currently, "sandbox-postgis" isn't part of the name of any running container, so let's spin one up.

In [7]:
!docker compose up -d

[+] Running 1/0
 ✔ Network 015_docker_postgres_sandbox_default      Created                0.0s 
 ⠿ Container 015_docker_postgres_sandbox-postgis-1  Starting               0.1s 
[+] Running 1/2
 ✔ Network 015_docker_postgres_sandbox_default      Created                0.0s 
 ⠿ Container 015_docker_postgres_sandbox-postgis-1  Starting               0.2s 
[+] Running 1/2
 ✔ Network 015_docker_postgres_sandbox_default      Created                0.0s 
 ⠿ Container 015_docker_postgres_sandbox-postgis-1  Starting               0.3s 
[+] Running 2/2
 ✔ Network 015_docker_postgres_sandbox_default      Created                0.0s 
 ✔ Container 015_docker_postgres_sandbox-postgis-1  Started                0.4s 


In [8]:
!docker ps -f name=sandbox-postgis

CONTAINER ID   IMAGE                    COMMAND                  CREATED         STATUS        PORTS                                         NAMES
90d06500a7f4   sandbox_postgis:15.3.3   "docker-entrypoint.s…"   2 seconds ago   Up 1 second   0.0.0.0:54321->5432/tcp, :::54321->5432/tcp   015_docker_postgres_sandbox-postgis-1


In [19]:
from contextlib import contextmanager
from typing import Union

import pandas as pd
import psycopg2


@contextmanager
def get_db_connection():
    conn = psycopg2.connect(
        dbname="db_name",
        user="db_username",
        password="db_password",
        host="localhost",
        port=54321
    )
    try:
        yield conn
    finally:
        conn.close()

def execute_query(query: str) -> Union[pd.DataFrame, None]:
    with get_db_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            rows = cur.fetchall()
            if rows:
                col_names = [desc[0] for desc in cur.description]
                return pd.DataFrame(rows, columns=col_names)

In [20]:
schemas = execute_query(query="""
    SELECT nspname AS schema_name
    FROM pg_namespace;
""")
schemas

,schema_name
0,pg_toast
1,pg_catalog
2,public
3,information_schema
4,topology
5,tiger
6,tiger_data


In [23]:
execute_query(query="""SELECT current_user;""")

,current_user
0,db_username


In [24]:
pg_databases_df = execute_query(
    query="""
        SELECT *
        FROM pg_database;
    """
)
pg_databases_df

,oid,datname,datdba,encoding,datlocprovider,datistemplate,datallowconn,datconnlimit,datfrozenxid,datminmxid,dattablespace,datcollate,datctype,daticulocale,datcollversion,datacl
0,5,postgres,10,6,c,False,True,-1,717,1,1663,en_US.utf8,en_US.utf8,None,2.31,None
1,16384,db_name,10,6,c,False,True,-1,717,1,1663,en_US.utf8,en_US.utf8,None,2.31,None
2,1,template1,10,6,c,True,True,-1,717,1,1663,en_US.utf8,en_US.utf8,None,2.31,"{=c/db_username,db_username=CTc/db_username}"
3,4,template0,10,6,c,True,False,-1,717,1,1663,en_US.utf8,en_US.utf8,None,None,"{=c/db_username,db_username=CTc/db_username}"
4,16385,template_postgis,10,6,c,True,True,-1,717,1,1663,en_US.utf8,en_US.utf8,None,2.31,None


In [25]:
pg_namespaces_df = execute_query(
    query="""
        SELECT *
        FROM pg_namespace;
    """
)
pg_namespaces_df

,oid,nspname,nspowner,nspacl
0,99,pg_toast,10,None
1,11,pg_catalog,10,"{db_username=UC/db_username,=U/db_username}"
2,2200,public,6171,"{pg_database_owner=UC/pg_database_owner,=U/pg_..."
3,13186,information_schema,10,"{db_username=UC/db_username,=U/db_username}"
4,19052,topology,10,"{db_username=UC/db_username,=U/db_username}"
5,19226,tiger,10,None
6,19482,tiger_data,10,None


In [26]:
pg_roles_df = execute_query(
    query="""
        SELECT *
        FROM pg_roles;
    """
)
pg_roles_df

,rolname,rolsuper,rolinherit,rolcreaterole,rolcreatedb,rolcanlogin,rolreplication,rolconnlimit,rolpassword,rolvaliduntil,rolbypassrls,rolconfig,oid
0,pg_database_owner,False,True,False,False,False,False,-1,********,None,False,None,6171
1,pg_read_all_data,False,True,False,False,False,False,-1,********,None,False,None,6181
2,pg_write_all_data,False,True,False,False,False,False,-1,********,None,False,None,6182
3,pg_monitor,False,True,False,False,False,False,-1,********,None,False,None,3373
4,pg_read_all_settings,False,True,False,False,False,False,-1,********,None,False,None,3374
5,pg_read_all_stats,False,True,False,False,False,False,-1,********,None,False,None,3375
6,pg_stat_scan_tables,False,True,False,False,False,False,-1,********,None,False,None,3377
7,pg_read_server_files,False,True,False,False,False,False,-1,********,None,False,None,4569
8,pg_write_server_files,False,True,False,False,False,False,-1,********,None,False,None,4570
9,pg_execute_server_program,False,True,False,False,False,False,-1,********,None,False,None,4571


In [27]:
pg_classes_df = execute_query(
    query="""
        SELECT *
        FROM pg_class;
    """
)
pg_classes_df

,oid,relname,relnamespace,reltype,reloftype,relowner,relam,relfilenode,reltablespace,relpages,...,relforcerowsecurity,relispopulated,relreplident,relispartition,relrewrite,relfrozenxid,relminmxid,relacl,reloptions,relpartbound
0,18312,geometry_dump,2200,18314,0,10,0,0,0,0,...,False,True,n,False,0,0,0,None,None,None
1,19439,idx_tiger_featnames_lname,19226,0,0,10,403,19439,0,1,...,False,True,n,False,0,0,0,None,None,None
2,18323,pg_toast_18319,99,0,0,10,2,18323,0,0,...,False,True,n,False,0,732,1,None,None,None
3,18324,pg_toast_18319_index,99,0,0,10,403,18324,0,1,...,False,True,n,False,0,0,0,None,None,None
4,19440,idx_tiger_featnames_tlid_statefp,19226,0,0,10,403,19440,0,1,...,False,True,n,False,0,0,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,19584,pagc_rules_id_seq,19226,0,0,10,0,19584,0,1,...,False,True,n,False,0,0,0,None,None,None
589,19590,pg_toast_19585,99,0,0,10,2,19590,0,0,...,False,True,n,False,0,735,1,None,None,None
590,19591,pg_toast_19585_index,99,0,0,10,403,19591,0,1,...,False,True,n,False,0,0,0,None,None,None
591,19592,pagc_rules_pkey,19226,0,0,10,403,19592,0,27,...,False,True,n,False,0,0,0,None,None,None


[^1] 